In [ ]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
df = pd.read_csv("train.csv")
dftest = pd.read_csv("test.csv")

<ipython-input-25-95eb98168068>:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train.csv")


In [ ]:
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [ ]:
min_age = 18
max_age = 100
max_num_credit_cards = 10
max_interest_rate = 50
max_num_loans = 10
max_num_credit_inquiries = 50

def find_latest(x):
    for value in reversed(x):
        if pd.notna(value):
            return value
    return None

def remove_underscore(value):
    if isinstance(value, str):
        return value.replace('_', '')
    return value

def check_valid_age(value):
    if value >= min_age and value <= max_age:
        return value
    return None

def check_num_credit_cards(value):
    return value.replace(0, 1, inplace=True)

def check_interest_rate(value):
    if value <= max_interest_rate:
        return value
    return None

def check_max_num_loan(value):
    if value >= 0 and value <= max_num_loans:
        return value
    return None

def check_num_delayed_payments(value):
    if isinstance(value, int) and value >= 0:
        return value
    return None

def check_num_credit_inquiries(value):
    if value <= max_num_credit_inquiries:
        return value
    return None

def check_outstanding_debt(value):
    if value >= 0:
        return value
    return None

def check_credit_history_age(value):
    try:
      years = int("".join(re.findall('[0-9]',''.join(value.split("and")[0]))))
      month = int("".join(re.findall('[0-9]',''.join(value.split("and")[1]))))
      return years*12 + month
    except:
      return None

def abs_num_bank_accounts(value):
  return value.apply(lambda x: abs(x))

def preprocessing(data):
    data = data.drop_duplicates()

    integer_columns = ['Age', 'Num_Bank_Accounts','Num_Credit_Card','Interest_Rate','Num_of_Loan','Delay_from_due_date',
                      'Num_of_Delayed_Payment','Num_Credit_Inquiries']
    float_columns = ['Annual_Income','Monthly_Inhand_Salary','Changed_Credit_Limit','Outstanding_Debt','Credit_Utilization_Ratio',
                    'Total_EMI_per_month','Amount_invested_monthly','Monthly_Balance']

    data = data.astype(str)
    data = data.applymap(remove_underscore)

    for column in integer_columns:
        data[column] = pd.to_numeric(data[column], errors='coerce').astype('Int64')

    for column in float_columns:
        data[column] = pd.to_numeric(data[column], errors='coerce').astype('float64')

    data['Age'] = data['Age'].apply(check_valid_age).astype('float64')
    data['Num_Credit_Card'] = data['Num_Credit_Card'].apply(check_num_credit_cards).astype('Int64')
    data['Num_Bank_Accounts'] = data['Num_Bank_Accounts'].apply(abs_num_bank_accounts).astype('Int64')
    data['Interest_Rate'] = data['Interest_Rate'].apply(check_interest_rate).astype('Int64')
    data['Num_of_Delayed_Payment'] = data['Num_of_Delayed_Payment'].apply(check_num_delayed_payments).astype('Int64')
    data['Num_Credit_Inquiries'] = data['Num_Credit_Inquiries'].notna().apply(check_num_credit_inquiries).astype('Int64') #FIX the boolean value issue
    #data['Outstanding_Debt'] = data['Outstanding_Debt'].apply(check_num_delayed_payments).astype('float64')
    data['Credit_History_Age'] = data['Credit_History_Age'].apply(check_credit_history_age).astype('Int64')
    return data

In [ ]:
del df['ID']
del df['Name']
del df['SSN']
df = preprocessing(df)
dftest = preprocessing(dftest)

In [ ]:
def group_data(data):
    grouped_data = df.groupby('Customer_ID').agg({
    'Name': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'Age': lambda x: x.iloc[-1] if not x.empty else None,
    'Occupation': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'Annual_Income': lambda x: x.iloc[-1] if not x.empty else None,
    'Monthly_Inhand_Salary': lambda x: x.iloc[-1] if not x.empty else None,
    'Interest_Rate': lambda x: x.iloc[-1] if not x.empty else None,
    'Num_of_Loan': lambda x: x.iloc[-1] if not x.empty else None,
    'Type_of_Loan': lambda x: x.iloc[-1] if not x.empty else None,
    'Delay_from_due_date': lambda x: x.dropna().mean(),
    'Num_of_Delayed_Payment': lambda x: x.dropna().sum(),
    'Num_Credit_Inquiries': lambda x: x.iloc[-1] if not x.empty else None,
    'Credit_Mix' : lambda x: x.iloc[-1] if not x.empty else None,
    'Outstanding_Debt': lambda x: x.iloc[-1] if not x.empty else None,
    'Credit_Utilization_Ratio': lambda x: x.iloc[-1] if not x.empty else None,
    'Credit_History_Age': lambda x: x.iloc[-1] if not x.empty else None,
    'Payment_of_Min_Amount': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'Total_EMI_per_month': lambda x: x.iloc[-1] if not x.empty else None,
    'Amount_invested_monthly': lambda x: x.dropna().mean(),
    'Payment_Behaviour': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'Monthly_Balance': lambda x: x.dropna().mean(),
    'Credit_Score': lambda x: x.iloc[-1] if not x.empty else None}).reset_index()

    return grouped_data

grouped_df = group_data(df)
grouped_dftest = group_data(dftest)
grouped_df

,Customer_ID,Name,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,CUS0x1000,Alistair Barrf,18,Lawyer,30625.940,2706.161667,27,2,"Credit-Builder Loan, and Home Equity Loan",62.250,...,Bad,1562.91,30.077191,129,Yes,42.941090,158.549735,HighspentMediumvaluepayments,335.375341,Poor
1,CUS0x1009,Arunah,26,Mechanic,52312.680,4250.390000,17,4,"Not Specified, Home Equity Loan, Credit-Builde...",7.250,...,,202.68,40.286997,372,Yes,108.366467,146.679378,HighspentMediumvaluepayments,428.743155,Standard
2,CUS0x100b,Shirboni,19,MediaManager,113781.390,9549.782500,1,0,nan,13.500,...,Good,1030.20,28.592943,190,No,0.000000,1861.759880,HighspentLargevaluepayments,781.229776,Standard
3,CUS0x1011,Schneyerh,44,Doctor,58918.470,5208.872500,17,3,"Student Loan, Credit-Builder Loan, and Debt Co...",27.250,...,Standard,473.14,27.829959,190,Yes,123.434939,1522.178300,LowspentMediumvaluepayments,332.642837,Standard
4,CUS0x1013,Cameront,44,Mechanic,98620.980,NaN,6,3,"Student Loan, Debt Consolidation Loan, and Per...",12.500,...,Good,1233.51,26.524864,214,No,228.018084,355.442408,HighspentMediumvaluepayments,472.781009,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS0xff3,Somervilled,55,Scientist,17032.785,1176.398750,2,3,"Personal Loan, Mortgage Loan, and Auto Loan",12.250,...,Good,1229.08,26.915577,<NA>,No,33.299764,83.918549,LowspentSmallvaluepayments,271.671562,Standard
12496,CUS0xff4,Poornimaf,37,Entrepreneur,25546.260,2415.855000,14,5,"Not Specified, Student Loan, Student Loan, Cre...",15.875,...,Standard,758.44,39.333488,225,Yes,101.328637,152.775690,LowspentSmallvaluepayments,259.981173,Standard
12497,CUS0xff6,Shieldsb,19,Doctor,117639.920,9727.326667,1,2,"Home Equity Loan, and Auto Loan",-0.125,...,Good,338.30,32.871853,299,No,126.638453,1990.195603,HighspentSmallvaluepayments,667.322417,Good
12498,CUS0xffc,Brads,18,Musician,60877.170,5218.097500,27,8,"Credit-Builder Loan, Payday Loan, Not Specifie...",45.375,...,,1300.13,29.026498,<NA>,Yes,272.809169,1386.651352,HighspentLargevaluepayments,339.951771,Poor


In [ ]:
grouped_dftest

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0.000000,8,0.000991,0.161638,0.787879,0.064557,989,0.984496,0.025743,1.0,1,0.312656,0.355045,0.305556,2,0.000524,0.032040,2,1.0,1
1,0.210526,10,0.001901,0.265532,0.484848,0.065823,2629,0.131783,0.241215,1.0,0,0.040486,0.718395,0.919192,2,0.001322,0.029385,2,1.0,2
2,0.026316,11,0.004481,0.622069,0.000000,0.063291,4449,0.228682,0.007594,1.0,2,0.206066,0.302223,0.459596,1,0.000000,0.412971,1,1.0,2
3,0.684211,4,0.002179,0.330018,0.484848,0.065190,4082,0.441860,0.011198,1.0,3,0.094603,0.275069,0.459596,2,0.001506,0.337022,5,1.0,2
5,0.236842,7,0.001676,0.230224,0.454545,0.063291,4449,0.143411,0.009396,1.0,3,0.068007,0.661284,0.628788,2,0.000000,0.055583,5,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12493,0.526316,4,0.003775,0.513362,0.212121,0.063291,4449,0.405039,0.015189,1.0,3,0.228982,0.601787,0.659091,1,0.000000,0.045228,1,1.0,2
12494,0.394737,9,0.001564,0.232250,0.151515,0.064557,4448,0.201550,0.002188,1.0,2,0.262497,0.471474,0.545455,1,0.000494,0.059771,5,1.0,0
12496,0.500000,6,0.000778,0.142107,0.393939,0.066456,2920,0.265504,0.012486,1.0,3,0.151689,0.684461,0.547980,2,0.001236,0.030749,6,1.0,2
12497,0.026316,4,0.004643,0.634013,0.000000,0.064557,1968,0.017442,0.003604,1.0,2,0.067623,0.454502,0.734848,1,0.001545,0.441697,3,1.0,0


In [ ]:
# Normalizing the dataset

def normalize_data(data):
    data.dropna(inplace=True)
    data.drop('Customer_ID', axis=1, inplace=True)
    data.drop('Name', axis=1, inplace=True)

    categorical_cols = ['Credit_Mix','Occupation','Type_of_Loan','Payment_of_Min_Amount','Payment_Behaviour', 'Credit_Score']

    data_cols = ['Age','Interest_Rate','Num_of_Loan','Delay_from_due_date',
             'Num_of_Delayed_Payment','Num_Credit_Inquiries','Annual_Income','Monthly_Inhand_Salary',
             'Outstanding_Debt','Credit_Utilization_Ratio','Credit_History_Age','Total_EMI_per_month',
             'Amount_invested_monthly','Monthly_Balance']

    label_encoder = LabelEncoder()
    scaler = MinMaxScaler()

    for column in categorical_cols:
        data[column] = label_encoder.fit_transform(data[column])

    for column in data_cols:
        data[column] = scaler.fit_transform(data[column].values.reshape(-1,1))

    return data

normalized_df = normalize_data(grouped_df)
normalized_dftest = normalize_data(grouped_dftest)

normalized_df

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0.000000,8,0.000991,0.161638,0.787879,0.064557,989,0.984496,0.025743,1.0,1,0.312656,0.355045,0.305556,2,0.000524,0.032040,2,1.0,1
1,0.210526,10,0.001901,0.265532,0.484848,0.065823,2629,0.131783,0.241215,1.0,0,0.040486,0.718395,0.919192,2,0.001322,0.029385,2,1.0,2
2,0.026316,11,0.004481,0.622069,0.000000,0.063291,4449,0.228682,0.007594,1.0,2,0.206066,0.302223,0.459596,1,0.000000,0.412971,1,1.0,2
3,0.684211,4,0.002179,0.330018,0.484848,0.065190,4082,0.441860,0.011198,1.0,3,0.094603,0.275069,0.459596,2,0.001506,0.337022,5,1.0,2
5,0.236842,7,0.001676,0.230224,0.454545,0.063291,4449,0.143411,0.009396,1.0,3,0.068007,0.661284,0.628788,2,0.000000,0.055583,5,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12493,0.526316,4,0.003775,0.513362,0.212121,0.063291,4449,0.405039,0.015189,1.0,3,0.228982,0.601787,0.659091,1,0.000000,0.045228,1,1.0,2
12494,0.394737,9,0.001564,0.232250,0.151515,0.064557,4448,0.201550,0.002188,1.0,2,0.262497,0.471474,0.545455,1,0.000494,0.059771,5,1.0,0
12496,0.500000,6,0.000778,0.142107,0.393939,0.066456,2920,0.265504,0.012486,1.0,3,0.151689,0.684461,0.547980,2,0.001236,0.030749,6,1.0,2
12497,0.026316,4,0.004643,0.634013,0.000000,0.064557,1968,0.017442,0.003604,1.0,2,0.067623,0.454502,0.734848,1,0.001545,0.441697,3,1.0,0


In [ ]:
normalized_dftest

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0.000000,8,0.000991,0.161638,0.787879,0.064557,989,0.984496,0.025743,1.0,1,0.312656,0.355045,0.305556,2,0.000524,0.032040,2,1.0,1
1,0.210526,10,0.001901,0.265532,0.484848,0.065823,2629,0.131783,0.241215,1.0,0,0.040486,0.718395,0.919192,2,0.001322,0.029385,2,1.0,2
2,0.026316,11,0.004481,0.622069,0.000000,0.063291,4449,0.228682,0.007594,1.0,2,0.206066,0.302223,0.459596,1,0.000000,0.412971,1,1.0,2
3,0.684211,4,0.002179,0.330018,0.484848,0.065190,4082,0.441860,0.011198,1.0,3,0.094603,0.275069,0.459596,2,0.001506,0.337022,5,1.0,2
5,0.236842,7,0.001676,0.230224,0.454545,0.063291,4449,0.143411,0.009396,1.0,3,0.068007,0.661284,0.628788,2,0.000000,0.055583,5,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12493,0.526316,4,0.003775,0.513362,0.212121,0.063291,4449,0.405039,0.015189,1.0,3,0.228982,0.601787,0.659091,1,0.000000,0.045228,1,1.0,2
12494,0.394737,9,0.001564,0.232250,0.151515,0.064557,4448,0.201550,0.002188,1.0,2,0.262497,0.471474,0.545455,1,0.000494,0.059771,5,1.0,0
12496,0.500000,6,0.000778,0.142107,0.393939,0.066456,2920,0.265504,0.012486,1.0,3,0.151689,0.684461,0.547980,2,0.001236,0.030749,6,1.0,2
12497,0.026316,4,0.004643,0.634013,0.000000,0.064557,1968,0.017442,0.003604,1.0,2,0.067623,0.454502,0.734848,1,0.001545,0.441697,3,1.0,0


In [ ]:
X, y = normalized_df.drop("Credit_Score",axis=1).values , normalized_df["Credit_Score"]
Xtesting, ytesting = normalized_dftest.drop("Credit_Score", axis=1).values, normalized_dftest["Credit_Score"]

In [ ]:
# Oversampling
from imblearn.over_sampling import SMOTE
sample = SMOTE(sampling_strategy='auto')
normalized_X, normalized_y = sample.fit_resample(X, y)
normailzed_Xtesting, normailized_ytesting = sample.fit_resample(Xtesting, ytesting)

In [ ]:
X_train = normalized_X
y_train = normalized_y
X_test = normailzed_Xtesting
y_test = normailized_ytesting

In [ ]:
# Implementing Logistic Regression

lr_model = LogisticRegression(penalty='l2',random_state=42)

# Train the model on the training data
lr_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = lr_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(accuracy)
print(conf_matrix)
print(classification_rep)

0.5309026233881725
[[3231  547  720]
 [ 884 2476 1138]
 [1600 1441 1457]]
              precision    recall  f1-score   support

           0       0.57      0.72      0.63      4498
           1       0.55      0.55      0.55      4498
           2       0.44      0.32      0.37      4498

    accuracy                           0.53     13494
   macro avg       0.52      0.53      0.52     13494
weighted avg       0.52      0.53      0.52     13494



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
